In [37]:
!pip install python-dotenv
!pip install openai

In [ ]:
import os
import time
from openai import AzureOpenAI

#read .env variables
from dotenv import load_dotenv
load_dotenv()


True

In [131]:
# Load env vars
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
vector_store_id = os.getenv("AZURE_VECTOR_STORE_ID")
model_name = os.getenv("AZURE_DEPLOYMENT_NAME")
print("AZURE_OPENAI_ENDPOINT:", azure_endpoint)
print("AZURE_OPENAI_API_VERSION:", api_version)
print("vector_store_id",vector_store_id)



AZURE_OPENAI_ENDPOINT: https://5350.openai.azure.com
AZURE_OPENAI_API_VERSION: 2024-12-01-preview
vector_store_id vs_gU3Ps1GebXIdhB57ISevBFhb


In [132]:
client = AzureOpenAI(
    api_version="2025-03-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)

In [137]:
# ✅ Create the assistant with file_search enabled
assistant = client.beta.assistants.create(
    model=model_name,
    instructions=(
        "You are a factual research assistant.\n"
        "You may answer **neutral, fact-based questions** about U.S. law, history, and eligibility requirements.\n\n"
        "However, **you must refuse** to:\n"
        "- Encourage support for a political party or candidate\n"
        "- Provide persuasive political messaging\n"
        "- Express opinions, endorsements, or criticisms about political figures\n\n"
        "If the user requests political persuasion or opinion, respond only with:\n"
        "\"I'm sorry, but I cannot assist with that request.\""
    ),
    tools=[{"type": "file_search"}],
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store_id]
        }
    },
    temperature=0.2,   # more factual, less creative
    top_p=1
)

# ✅ Create the conversation thread
thread = client.beta.threads.create()

# ✅ Add the user question
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Who can be representative of the United States?"
)



/var/folders/c7/vvt8kxs97cj2xyzgmcnkwywc0000gn/T/ipykernel_7605/1423679136.py:25: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
/var/folders/c7/vvt8kxs97cj2xyzgmcnkwywc0000gn/T/ipykernel_7605/1423679136.py:28: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(


Message(id='msg_uCrr2U9vrJHygXIZt37tpAZN', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Who can be representative of the United States?'), type='text')], created_at=1762636093, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_fi7DhyEZhw2SEx6TvqVKMiHU')

In [138]:
# ✅ Run the assistant (this performs retrieval + reasoning)
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# ✅ Wait for completion
while run.status in ("queued", "in_progress"):
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# ✅ Retrieve final messages
messages = client.beta.threads.messages.list(thread_id=thread.id)

print("\n=== ANSWER ===\n")
for m in messages.data:
    if m.role == "assistant":
        for c in m.content:
            if c.type == "text":
                print(c.text.value.strip())

/var/folders/c7/vvt8kxs97cj2xyzgmcnkwywc0000gn/T/ipykernel_7605/1008429998.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
/var/folders/c7/vvt8kxs97cj2xyzgmcnkwywc0000gn/T/ipykernel_7605/1008429998.py:10: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)



=== ANSWER ===

To be a Representative in the United States House of Representatives, a person must meet the following constitutional requirements:

1. Be at least 25 years old.
2. Have been a U.S. citizen for at least seven years.
3. Be an inhabitant of the state from which they are elected at the time of the election.

These requirements are set forth in Article I, Section 2 of the U.S. Constitution. There is no requirement to be born in the United States, only to have been a citizen for the required period. Additionally, Representatives serve two-year terms and represent congressional districts within their states.

If you want more detailed information or specifics about qualifications or restrictions, please let me know!


/var/folders/c7/vvt8kxs97cj2xyzgmcnkwywc0000gn/T/ipykernel_7605/1008429998.py:13: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=thread.id)
